Note to self: activate conda ai102

In [2]:
!pip install opencv-python
!pip install pillow
!pip install python-dotenv

  Using cached opencv_python-4.11.0.86-cp37-abi3-win_amd64.whl.metadata (20 kB)
Using cached opencv_python-4.11.0.86-cp37-abi3-win_amd64.whl (39.5 MB)
   ---------------------------------------- 0.0/15.9 MB ? eta -:--:--
   -- ------------------------------------- 1.0/15.9 MB 5.6 MB/s eta 0:00:03
   ----- ---------------------------------- 2.4/15.9 MB 5.8 MB/s eta 0:00:03
   --------- ------------------------------ 3.7/15.9 MB 5.7 MB/s eta 0:00:03
   ------------- -------------------------- 5.2/15.9 MB 6.1 MB/s eta 0:00:02
   ----------------- ---------------------- 6.8/15.9 MB 6.5 MB/s eta 0:00:02
   --------------------- ------------------ 8.7/15.9 MB 6.9 MB/s eta 0:00:02
   --------------------------- ------------ 10.7/15.9 MB 7.3 MB/s eta 0:00:01
   -------------------------------- ------- 13.1/15.9 MB 7.8 MB/s eta 0:00:01
   -------------------------------------- - 15.5/15.9 MB 8.2 MB/s eta 0:00:01
   ---------------------------------------- 15.9/15.9 MB 8.1 MB/s eta 0:00:00
   --

In [ ]:
!pip install promptflow-core


   ---------------------------------------- 0.0/987.9 kB ? eta -:--:--
   ------------------------------- -------- 786.4/987.9 kB 4.2 MB/s eta 0:00:01
   ---------------------------------------- 987.9/987.9 kB 3.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/894.2 kB ? eta -:--:--
   ---------------------------------------- 894.2/894.2 kB 5.1 MB/s eta 0:00:00


In [ ]:
!pip install azure-ai-textanalytics==5.3.0
!pip install azure-ai-vision-imageanalysis==1.0.0b3
!pip install azure-ai-formrecognizer==3.3.3
!pip install openai
!pip install azure-cognitiveservices-speech
!pip install azure.identity 

!pip install azure-cognitiveservices-vision-computervision


In [2]:
import cv2
import time
from PIL import Image
import numpy as np
import os

# Folder
folder = "frames"

# Create the frames folder if it doesn't exist
frames_dir = os.path.join(os.getcwd(), folder)
os.makedirs(frames_dir, exist_ok=True)

# Initialize the webcam
cap = cv2.VideoCapture(0)

# Check if the webcam is opened correctly
if not cap.isOpened():
    raise IOError("Cannot open webcam")

# Wait for the camera to initialize and adjust light levels
time.sleep(2)

while True:
    ret, frame = cap.read()
    if ret:
        # Convert the frame to a PIL image
        pil_img = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

        # Resize the image
        max_size = 250
        ratio = max_size / max(pil_img.size)
        new_size = tuple([int(x*ratio) for x in pil_img.size])
        resized_img = pil_img.resize(new_size, Image.LANCZOS)

        # Convert the PIL image back to an OpenCV image
        frame = cv2.cvtColor(np.array(resized_img), cv2.COLOR_RGB2BGR)

        # Save the frame as an image file
        print("📸 Say cheese! Saving frame.")
        path = f"{folder}/frame.jpg"
        cv2.imwrite(path, frame)
    else:
        print("Failed to capture image")

    # Wait for 2 seconds
    time.sleep(2)

# Release the camera and close all windows
cap.release()
cv2.destroyAllWindows()

📸 Say cheese! Saving frame.
📸 Say cheese! Saving frame.


KeyboardInterrupt: 

In [ ]:
import os
import base64
import time
import cv2
from PIL import Image
import numpy as np
import azure.cognitiveservices.speech as speechsdk
from openai import AzureOpenAI 
from azure.ai.vision.imageanalysis import ImageAnalysisClient
from azure.ai.vision.imageanalysis.models import VisualFeatures
from azure.core.credentials import AzureKeyCredential
from dotenv import load_dotenv

load_dotenv()

# Azure AI Services configuration
AZURE_AI_KEY = os.environ.get("AI_SERVICE_KEY")
AZURE_AI_ENDPOINT = os.environ.get("AI_SERVICE_ENDPOINT")
OAI_SERVICE_ENDPOINT = os.environ.get("OAI_SERVICE_ENDPOINT")
OAI_API_KEY = os.environ.get("OAI_API_KEY")
OAI_DEPLOYMENT= os.environ.get("OAI_DEPLOYMENT_NAME")
VISION_ENDPOINT = os.environ.get("VISION_SERVICE_ENDPOINT")
VISION_API_KEY = os.environ.get("VISION_API_KEY")

REGION = os.environ.get('LOCATION')

speech_config = speechsdk.SpeechConfig(subscription=AZURE_AI_KEY, region=REGION)


speech_config.speech_synthesis_voice_name='en-GB-EthanNeural'
audio_config = speechsdk.audio.AudioOutputConfig(use_default_speaker=True)
speech_synthesizer = speechsdk.SpeechSynthesizer(speech_config=speech_config, audio_config=audio_config)

oai_client = AzureOpenAI(  
    azure_endpoint=OAI_SERVICE_ENDPOINT,  
    api_key=OAI_API_KEY,  
    api_version="2025-01-01-preview",
)

cv_client = ImageAnalysisClient(
    endpoint=VISION_ENDPOINT,
    credential=AzureKeyCredential(VISION_API_KEY)
)

# Function to analyze an image using Azure OpenAI
def analyze_image(image_path):
    with open(image_path, "rb") as image_file:
        # use computer vision to generate a description of the image
        # Initialize the Computer Vision client
        vision_result = cv_client.analyze(
        image_data=image_file,
        visual_features=[
            VisualFeatures.CAPTION,
            ],
    )
        #print(vision_result.caption.text)
        description = vision_result.caption.text if vision_result.caption is not None else "I am currently blindfolded and cannot see anything."
        
        # Use OpenAI to generate a description
        prompt = [
            {"role": "system", "content": "You are a renowned British documentary narrator known for your eloquence and vivid descriptions. Based on an image provided, deliver a rich, engaging narrative that paints a vivid picture to the audience. Your descriptions should evoke emotions, draw upon relatable metaphors or analogies, and reflect curiosity and awe for the natural or human-made wonders depicted. If there is a Woman in the image, know that her name is Victoria and she's a technical trainer delivering training on artificial intelligence and using AI computer vision, OPEN AI to create this narration. "},
            {"role": "user", "content": f"Describe the content of this image: {description}"}
         ]
        response = oai_client.chat.completions.create(
            model = OAI_DEPLOYMENT,
            messages=prompt,
            max_tokens=100,
            temperature=0.7,
            
        )
        # Extract the generated description from the response
        return response.choices[0].message.content


# Function to play audio using Azure AI Services
def play_audio(text):
    print("🎙️ Azure says:")
    print(text)
    speech_synthesizer.speak_text_async(text).get()

# Main function
def main():
    folder = "frames"
    frames_dir = os.path.join(os.getcwd(), folder)
    os.makedirs(frames_dir, exist_ok=True)

    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        raise IOError("Cannot open webcam")

    time.sleep(2)

    while True:
        ret, frame = cap.read()
        if ret:
            pil_img = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
            max_size = 250
            ratio = max_size / max(pil_img.size)
            new_size = tuple([int(x * ratio) for x in pil_img.size])
            resized_img = pil_img.resize(new_size, Image.LANCZOS)
            frame = cv2.cvtColor(np.array(resized_img), cv2.COLOR_RGB2BGR)

            path = os.path.join(frames_dir, "frame.jpg")
            cv2.imwrite(path, frame)

            print("👀 Analyzing image...")
            description = analyze_image(path)
            play_audio(description)

        else:
            print("Failed to capture image")

        time.sleep(5)

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()

👀 Analyzing image...
🎙️ Azure says:
In the heart of a room adorned with the soft luminescence of knowledge, we find Victoria, a beacon of intellect and innovation, immersed in the ethereal dance of artificial intelligence. As she sits poised at the nexus of human ingenuity and machine learning, her gaze fixed upon the digital tapestry before her, one cannot help but marvel at the seamless fusion of technology and intellect that unfolds in this sacred space.

The room itself seems to pulsate with a rhythmic energy, a harmonious blend
👀 Analyzing image...
🎙️ Azure says:
In the heart of a technologically advanced chamber, where the hum of innovation reverberates off the sleek walls, we find Victoria, a luminary in the realm of artificial intelligence. With a countenance of focused determination, she sits at the nexus of knowledge and possibility, her gaze fixed upon the digital canvas before her.

As she delves into the intricate world of AI, her deft fingers dance across the keyboard lik